# Database 

In [123]:
import pandas as pd
df = pd.read_excel("US 2019 Faculty Evaluation.xlsx")

# Lets do some basic inspection of the data
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 18 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Faculty ID                                             719 non-null    int64  
 1   Highest Degree of Faculty                              523 non-null    object 
 2   Title of Faculty                                       716 non-null    object 
 3   Full Name of Faculty                                   719 non-null    object 
 4   Total Number of Students in the Class                  719 non-null    int64  
 5   Total Respondents                                      719 non-null    int64  
 6   % of Respondents                                       718 non-null    float64
 7   Mean Score                                             719 non-null    float64
 8   % Score                                           

ok so basically im going to do the database design in notebook. Dont ask why, im just way to comfortable with python, but the doccumentation has the detial steps for how all of this works. so trust me on this, it will make sense in the end

# Faculty table

In [124]:
# Ok so we have the columns and now we are going to make the tables for the database

# First up is the faculty table
faculty_cols = [
    "Faculty ID", "Full Name of Faculty", "Title of Faculty",
    "Highest Degree of Faculty", "Are they Adjunct (Y/N)",
    "Total Number of Classes Being Taught by the Faculty",
    "Total Number of Students  Being Taught by the Faculty"
]

faculty_df = df[faculty_cols].drop_duplicates(subset=["Faculty ID"]).reset_index(drop=True)

# Courses table

In [125]:
# next up is the course table
course_cols = ["Course Code", "Course Description", "School", "Program"]

courses_df = df[course_cols].drop_duplicates(subset=["Course Code"]).reset_index(drop=True)

# Sections tables

In [126]:
# Sections table
sections_df = df[["Course Code", "Section Code"]].drop_duplicates()
sections_df["section_id"] = sections_df["Course Code"] + "_" + sections_df["Section Code"]
sections_df["semester"] = "US2019"  # add manually or infer

# Teaching Assingments table

In [127]:
# the assingments table, basically the link between faculty and sections
assignments_df = df[["Faculty ID", "Course Code", "Section Code"]].copy()
assignments_df["section_id"] = assignments_df["Course Code"] + "_" + assignments_df["Section Code"]
assignments_df = assignments_df.drop(columns=["Course Code", "Section Code"])

# Evaluation table

In [128]:
import sqlite3

# Create or connect to the database file
conn = sqlite3.connect("faculty_evaluation.db")

# Reload the original DataFrame from the Excel file to ensure it has the correct columns
df = pd.read_excel("US 2019 Faculty Evaluation.xlsx")

eval_cols = [
    "Faculty ID", "Full Name of Faculty",
    "Course Code", "Section Code",
    "Total Number of Students in the Class",
    "Total Respondents", "% of Respondents",
    "Mean Score", "% Score", "Letter Grade"
]

evaluations_df = df[eval_cols].copy()

# Build section_id
evaluations_df["section_id"] = evaluations_df["Course Code"] + "_" + evaluations_df["Section Code"]

# Rename to match SQL schema
evaluations_df = evaluations_df.rename(columns={
    "Faculty ID": "faculty_id",
    "Full Name of Faculty": "full_name",
    "Course Code": "course_code",
    "Section Code": "section_code",
    "Total Number of Students in the Class": "total_students",
    "Total Respondents": "total_respondents",
    "% of Respondents": "response_rate",
    "Mean Score": "mean_score",
    "% Score": "percent_score",
    "Letter Grade": "letter_grade"
})

# Insert into SQL
evaluations_df.to_sql("Evaluations", conn, if_exists="append", index=False)

719

# Departments table

In [187]:
cursor.executescript("""
DROP TABLE IF EXISTS Schools;
DROP TABLE IF EXISTS Courses;

CREATE TABLE Schools (
    school_id TEXT PRIMARY KEY,
    school_name TEXT
);

CREATE TABLE Courses (
    course_code TEXT PRIMARY KEY,
    school_id TEXT,
    FOREIGN KEY(school_id) REFERENCES Schools(school_id)
);
""")
conn.commit()

In [188]:
# Clean column names just in case
courses_df.columns = courses_df.columns.str.strip()

# Build Schools table
school_df = courses_df[["school"]].drop_duplicates().copy()
school_df["school_id"] = school_df["school"].str.replace(" ", "_").str.lower()
school_df = school_df.rename(columns={"school": "school_name"})
school_df.to_sql("Schools", conn, if_exists="append", index=False)

# Build Courses table
course_df = courses_df[["course_code", "school"]].drop_duplicates().copy()
course_df["school_id"] = course_df["school"].str.replace(" ", "_").str.lower()
course_df = course_df[["course_code", "school_id"]]
course_df.to_sql("Courses", conn, if_exists="append", index=False)

IntegrityError: UNIQUE constraint failed: Schools.school_id

ayt so everything is as it should be. So we are linking all of this to SQLite as the database, and we have to load them in

# SQLite

In [129]:
# So we first have to import the library so that we can work with the database
import sqlite3

# Create or connect to the database file
conn = sqlite3.connect("faculty_evaluation.db")
cursor = conn.cursor()

# Worked like a charm, now we use the schema to create the tables in the database
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Faculty (
    faculty_id INTEGER PRIMARY KEY,
    full_name TEXT NOT NULL,
    title TEXT,
    highest_degree TEXT,
    adjunct_flag TEXT CHECK(adjunct_flag IN ('Y','N')),
    total_classes_taught INTEGER,
    total_students_taught INTEGER
);

CREATE TABLE IF NOT EXISTS Courses (
    course_code TEXT PRIMARY KEY,
    course_description TEXT,
    school TEXT,
    program_level TEXT CHECK(program_level IN ('UNDG','GRAD','DOC'))
);

CREATE TABLE IF NOT EXISTS Sections (
    section_id TEXT PRIMARY KEY,
    course_code TEXT NOT NULL,
    section_code TEXT NOT NULL,
    semester TEXT,
    FOREIGN KEY(course_code) REFERENCES Courses(course_code)
);

CREATE TABLE IF NOT EXISTS TeachingAssignments (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    faculty_id INTEGER NOT NULL,
    section_id TEXT NOT NULL,
    role TEXT DEFAULT 'Lead Instructor',
    FOREIGN KEY(faculty_id) REFERENCES Faculty(faculty_id),
    FOREIGN KEY(section_id) REFERENCES Sections(section_id)
);

CREATE TABLE IF NOT EXISTS Evaluations (
    evaluation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    section_id TEXT NOT NULL,
    total_students INTEGER,
    total_respondents INTEGER,
    response_rate REAL,
    mean_score REAL,
    percent_score REAL,
    letter_grade TEXT,
    FOREIGN KEY(section_id) REFERENCES Sections(section_id)
);
""")
conn.commit()


# No issues so far so good


In [130]:
cursor.executescript("""
DROP TABLE IF EXISTS Evaluations;

CREATE TABLE Evaluations (
    evaluation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    faculty_id INTEGER,
    full_name TEXT,
    course_code TEXT,
    section_code TEXT,
    section_id TEXT NOT NULL,
    total_students INTEGER,
    total_respondents INTEGER,
    response_rate REAL,
    mean_score REAL,
    percent_score REAL,
    letter_grade TEXT,
    FOREIGN KEY(faculty_id) REFERENCES Faculty(faculty_id)
);
""")
conn.commit()

yeah so had to change this because of some issues with functions 

# Loading the dataframe into the tables

In [131]:
# Next is we load the dataframes into their respective tables
# we will use pandas to_sql function for this

# Rename columns to match the database schema
faculty_df = faculty_df.rename(columns={
    "Faculty ID": "faculty_id",
    "Full Name of Faculty": "full_name",
    "Title of Faculty": "title",
    "Highest Degree of Faculty": "highest_degree",
    "Are they Adjunct (Y/N)": "adjunct_flag",
    "Total Number of Classes Being Taught by the Faculty": "total_classes_taught",
    "Total Number of Students  Being Taught by the Faculty": "total_students_taught"
})

courses_df = courses_df.rename(columns={
    "Course Code": "course_code",
    "Course Description": "course_description",
    "School": "school",
    "Program": "program_level"
})

sections_df = sections_df.rename(columns={
    "Course Code": "course_code",
    "Section Code": "section_code"
})

assignments_df = assignments_df.rename(columns={
    "Faculty ID": "faculty_id"
})

evaluations_df = evaluations_df.rename(columns={
    "Total Number of Students in the Class": "total_students",
    "Total Respondents": "total_respondents",
    "% of Respondents": "response_rate",
    "Mean Score": "mean_score",
    "% Score": "percent_score",
    "Letter Grade": "letter_grade"
})


# Use replace so reruns rebuild tables cleanly
faculty_df.to_sql("Faculty", conn, if_exists="replace", index=False)
courses_df.to_sql("Courses", conn, if_exists="replace", index=False)
sections_df.to_sql("Sections", conn, if_exists="replace", index=False)
assignments_df.to_sql("TeachingAssignments", conn, if_exists="replace", index=False)
evaluations_df.to_sql("Evaluations", conn, if_exists="replace", index=False)

# --- Verification block ---
print(" Database rebuilt successfully. Current tables and row counts:")

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
for t in tables["name"]:
    count = pd.read_sql(f"SELECT COUNT(*) AS cnt FROM {t};", conn)["cnt"].iloc[0]
    print(f"- {t}: {count} rows")

 Database rebuilt successfully. Current tables and row counts:
- sqlite_sequence: 1 rows
- Deans: 5 rows
- FeedbackCorpus: 22698 rows
- Schools: 0 rows
- Faculty: 301 rows
- Courses: 438 rows
- Sections: 719 rows
- TeachingAssignments: 719 rows
- Evaluations: 719 rows


# Verifying the inserts

In [132]:
# This is some basic verification to see if the data was loaded correctly
print(pd.read_sql("SELECT * FROM Faculty LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Courses LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Evaluations LIMIT 5;", conn))

   faculty_id                        full_name title  \
0      600716           Mohamed, Mwanashehe S.    MS   
1      620040           Owuor, John David_Ouma    MR   
2      620489  Gitahi, Jesse Elikanah_Machirah    MR   
3      657923                  Odoyo, Fredrick    DR   
4      611121            Gatumo, Francis Mambo    MR   

             highest_degree adjunct_flag  total_classes_taught  \
0  Master of Business Admin            Y                     2   
1  Master of Business Admin            Y                     3   
2  Master of Business Admin            Y                     4   
3  Doctor of Business Admin            N                     1   
4  Master of Business Admin            N                     4   

   total_students_taught  
0                     75  
1                    132  
2                    123  
3                     38  
4                     94  
  course_code course_description                    school program_level
0     ACT1010         Accountin

In [133]:
# One more verification never hurts
print(pd.read_sql("SELECT COUNT(*) AS faculty_count FROM Faculty;", conn))
print(pd.read_sql("SELECT COUNT(*) AS course_count FROM Courses;", conn))
print(pd.read_sql("SELECT COUNT(*) AS section_count FROM Sections;", conn))
print(pd.read_sql("SELECT COUNT(*) AS assignment_count FROM TeachingAssignments;", conn))
print(pd.read_sql("SELECT COUNT(*) AS evaluation_count FROM Evaluations;", conn))

# Spot check a few rows
print(pd.read_sql("SELECT * FROM Faculty LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Courses LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Sections LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM TeachingAssignments LIMIT 5;", conn))
print(pd.read_sql("SELECT * FROM Evaluations LIMIT 5;", conn))


   faculty_count
0            301
   course_count
0           438
   section_count
0            719
   assignment_count
0               719
   evaluation_count
0               719
   faculty_id                        full_name title  \
0      600716           Mohamed, Mwanashehe S.    MS   
1      620040           Owuor, John David_Ouma    MR   
2      620489  Gitahi, Jesse Elikanah_Machirah    MR   
3      657923                  Odoyo, Fredrick    DR   
4      611121            Gatumo, Francis Mambo    MR   

             highest_degree adjunct_flag  total_classes_taught  \
0  Master of Business Admin            Y                     2   
1  Master of Business Admin            Y                     3   
2  Master of Business Admin            Y                     4   
3  Doctor of Business Admin            N                     1   
4  Master of Business Admin            N                     4   

   total_students_taught  
0                     75  
1                    132  
2    

This fantastic checkpoint, the counts and sample rows confirm that the ETL pipeline worked exactly as intended:
• 	Faculty count = 301 deduplication succeeded .
• 	Course count = 438  unique course codes extracted.
• 	Section count = 719 matches the number of Excel rows, since each row defined a unique section.
• 	Assignments = 719  one per Excel row, linking faculty ↔ section.
• 	Evaluations = 719 one per Excel row, storing metrics.
And the sample rows show the normalization is clean: faculty metadata, course metadata, section IDs, assignments, and evaluations are all aligned.

What This Means
•   SQLite database is now populated with normalized tables.
• 	Auto‑increment IDs (, ) are working.
• 	Foreign keys are linking everything together.
•   I now query across UG, GRAD, and DOC seamlessly.


ok here i just want to do basic query tests on the database

In [134]:
# Basic queery tests on the DB
def get_top_positive_lecturer(course_code):
    query = """
    SELECT f.full_name, AVG(e.mean_score) AS avg_score
    FROM Evaluations e
    JOIN Sections s ON e.section_id = s.section_id
    JOIN TeachingAssignments t ON e.section_id = t.section_id
    JOIN Faculty f ON t.faculty_id = f.faculty_id
    WHERE s.course_code = ?
    GROUP BY f.full_name
    ORDER BY avg_score DESC
    LIMIT 1;
    """
    return pd.read_sql(query, conn, params=[course_code])

# Here i wann use ACT1010 as a test case since it has 3 different lecturers

result = get_top_positive_lecturer("ACT1010")
print(result)

                full_name  avg_score
0  Owuor, John David_Ouma       4.23


something that is odd is when looking at the excel sheet we have Muhammed having a slightly higher mean score and percentage. But its fine since Muhammed teaches two sections teh queery did an average of the two and thats why John is above him.

In [135]:
# Ok so here we are doing it by section 
def rank_lecturers_by_best_section(course_code):
    query = """
    SELECT f.full_name, s.section_code, e.mean_score
    FROM Evaluations e
    JOIN Sections s ON e.section_id = s.section_id
    JOIN TeachingAssignments t ON e.section_id = t.section_id
    JOIN Faculty f ON t.faculty_id = f.faculty_id
    WHERE s.course_code = ?
    ORDER BY e.mean_score DESC;
    """
    return pd.read_sql(query, conn, params=[course_code])
result = rank_lecturers_by_best_section("ACT1010")
print(result)

                         full_name section_code  mean_score
0           Mohamed, Mwanashehe S.            A        4.28
1           Owuor, John David_Ouma            B        4.23
2           Mohamed, Mwanashehe S.            D        4.17
3  Gitahi, Jesse Elikanah_Machirah            C        4.04


yup yup yup that works great.So i wanna run like 5 more test querries to ensure everything is fine and all that, then ill push it to the streamlit app

In [136]:
# Lets do top course lecturers by section
rank_lecturers_by_best_section("BUS3010")

,full_name,section_code,mean_score
0,"Kibet, Eric M.",C,4.44
1,"Kibet, Eric M.",A,4.43
2,"Akuma, Samson Mainye",B,4.43
3,"Kibet, Eric M.",D,4.41


In [137]:
# Ayt so here we want to get the top lecturers by school of business
query = """
SELECT f.full_name, c.course_code, s.section_code, e.mean_score, e.letter_grade
FROM Evaluations e
JOIN Sections s ON e.section_id = s.section_id
JOIN Courses c ON s.course_code = c.course_code
JOIN TeachingAssignments t ON e.section_id = t.section_id
JOIN Faculty f ON t.faculty_id = f.faculty_id
WHERE c.school = ?
ORDER BY e.mean_score DESC;
"""
pd.read_sql(query, conn, params=["Chandaria School of Busi"])


,full_name,course_code,section_code,mean_score,letter_grade
0,"Kamau, Joseph",GMB6020,A,4.71,A
1,"Macharia, Jimmy",MBH6040,A,4.71,A
2,"Karimi, James Mark,, Ngari",MBH6070,A,4.63,A
3,"Achoki, George",ACT6010,B,4.62,A
4,"Achoki, George",DBA7030,A,4.60,A
...,...,...,...,...,...
215,"Miriti, Gilbert Mugambi",BUS4010,A,3.90,B-
216,"Joseph, Muchina",BUS5080,A,3.73,C+
217,"Kamuri, Simon",ENT1010,B,3.64,C
218,"Njenga, Edward Gachangi",BUS6225,C,3.55,C


lovely scenes and it matches accuratelty with the excel sheet

In [138]:
# ok so here i want to see the bottom lectures in the school of business based on mean score
query = """
SELECT f.full_name, c.course_code, s.section_code, e.mean_score, e.letter_grade
FROM Evaluations e
JOIN Sections s ON e.section_id = s.section_id
JOIN Courses c ON s.course_code = c.course_code
JOIN TeachingAssignments t ON e.section_id = t.section_id
JOIN Faculty f ON t.faculty_id = f.faculty_id
WHERE c.school = ?
ORDER BY e.mean_score ASC;
"""
pd.read_sql(query, conn, params=["Chandaria School of Busi"])


,full_name,course_code,section_code,mean_score,letter_grade
0,"Ogada, Agnes Owuor",FIN6000,C,3.33,C-
1,"Njenga, Edward Gachangi",BUS6225,C,3.55,C
2,"Kamuri, Simon",ENT1010,B,3.64,C
3,"Joseph, Muchina",BUS5080,A,3.73,C+
4,"Miriti, Gilbert Mugambi",BUS4010,A,3.90,B-
...,...,...,...,...,...
215,"Adan, Ali",GMB6055,A,4.60,A
216,"Achoki, George",ACT6010,B,4.62,A
217,"Karimi, James Mark,, Ngari",MBH6070,A,4.63,A
218,"Kamau, Joseph",GMB6020,A,4.71,A


ok the querries do work as they should so we can now move


Ok so had to comeback and create a deans table

In [139]:
# Create Deans table
conn.execute("""
CREATE TABLE IF NOT EXISTS Deans (
    faculty_id TEXT PRIMARY KEY,
    school TEXT NOT NULL
);
""")

# Example insertions (you can expand this later)
deans_data = [
    ("639355", "Chandaria School of Business"),
    ("262921", "School of Humanities and Social Sciences"),
    ("658751", "School of Science and Technology"),
    ('651571', "School Of Pharmacy"),
    ('659487', "School of Science")
]

conn.executemany("INSERT OR REPLACE INTO Deans (faculty_id, school) VALUES (?, ?);", deans_data)
conn.commit()

# Verify
pd.read_sql("SELECT * FROM Deans;", conn)

,faculty_id,school
0,639355,Chandaria School of Business
1,262921,School of Humanities and Social Sciences
2,658751,School of Science and Technology
3,651571,School Of Pharmacy
4,659487,School of Science


# Comments Table additions

Alright im back here again to add in the already parsed undergradute data so that we can start to build up on its knowledge base. Currently this is only for the undergradute level. i Still need to go back and parse data for the docotral levle and the masters level. Well get to that when we get to it.

In [140]:
# Alright so first we gotta load in the excel file that has the comments feedback
import pandas as pd
import sqlite3
# This is to normalize the names for matching since its the only unique identfieir we have
def normalize_name(name):
    return (
        str(name).strip().lower()
        .replace(".", "")
        .replace(",", "")
    )

# Ok so we have the NA issue again in vs code so we gotta takle that like we did last time
# Load Excel
excel_df = pd.read_excel("faculty_evaluations_cleaned.xlsx", keep_default_na=False)
# Next lets view the first 10 rows to understand its structure
excel_df.head(10)
# Alright i viewd the data in data wrangler and there was no missing comment so thats enough



,Course Code,Section Code,Instructor,Semester,Comment Type,Comment Text
0,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,N/A
1,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,ok
2,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,n/a
3,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good
4,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,Good
5,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,good
6,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,N/A
7,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,.
8,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Strengths,PowerPoints .
9,IST3005,A,"Afundi, Patrick",Summer Semester 2019 (UNDG),Weaknesses,n/a


In [141]:
# I just wanna printout the names of the columns in the file so that i can use them later
print(excel_df.columns.tolist())


['Course Code', 'Section Code', 'Instructor', 'Semester', 'Comment Type', 'Comment Text']


In [142]:
# Next we normalize the instructer names
excel_df["Normalized Name"] = excel_df["Instructor"].apply(normalize_name)

Alright columns now match appriopraitley so now its time to start joining the data

In [143]:
# First step is to laod the faculty table from the database to get the normalized names there too
faculty_df = pd.read_sql("SELECT faculty_id, full_name FROM Faculty", conn)
faculty_df["Normalized Name"] = faculty_df["full_name"].apply(normalize_name)

In [144]:
# Next is merging the excel to the faculty id
merged_df = pd.merge(
    excel_df,
    faculty_df[["faculty_id", "Normalized Name"]],
    on="Normalized Name",
    how="left"
)

In [145]:
# Next i wann a create a feedback corpus table in the database to hold the comments
cursor.executescript("""
DROP TABLE IF EXISTS FeedbackCorpus;

CREATE TABLE FeedbackCorpus (
    feedback_id INTEGER PRIMARY KEY AUTOINCREMENT,
    faculty_id INTEGER,
    instructor TEXT,
    course_code TEXT,
    section_code TEXT,
    semester TEXT,
    comment_type TEXT,
    comment_text TEXT,
    FOREIGN KEY(faculty_id) REFERENCES Faculty(faculty_id)
);
""")
conn.commit()

In [146]:
# Then lastly i wann place in the data
# The faculty_id column is already present in the table from its creation, so no need to add it again

# Rename columns to match the FeedbackCorpus table and select only the columns that exist in the table
insert_df = merged_df.rename(columns={
    "Course Code": "course_code",
    "Section Code": "section_code",
    "Semester": "semester",
    "Comment Type": "comment_type",
    "Comment Text": "comment_text",
    "Instructor": "instructor"
})

insert_cols = ["faculty_id", "instructor", "course_code", "section_code", "semester", "comment_type", "comment_text"]
insert_df = insert_df[[c for c in insert_cols if c in insert_df.columns]]

insert_df.to_sql("FeedbackCorpus", conn, if_exists="append", index=False)

11349

In [147]:
# Alright next i just need to validate that the data was inserted correctly
df = pd.read_sql("SELECT * FROM FeedbackCorpus LIMIT 10", conn)
print(df)


   feedback_id faculty_id       instructor course_code section_code  \
0            1       None  Afundi, Patrick     IST3005            A   
1            2       None  Afundi, Patrick     IST3005            A   
2            3       None  Afundi, Patrick     IST3005            A   
3            4       None  Afundi, Patrick     IST3005            A   
4            5       None  Afundi, Patrick     IST3005            A   
5            6       None  Afundi, Patrick     IST3005            A   
6            7       None  Afundi, Patrick     IST3005            A   
7            8       None  Afundi, Patrick     IST3005            A   
8            9       None  Afundi, Patrick     IST3005            A   
9           10       None  Afundi, Patrick     IST3005            A   

                      semester comment_type   comment_text  
0  Summer Semester 2019 (UNDG)    Strengths            N/A  
1  Summer Semester 2019 (UNDG)    Strengths             ok  
2  Summer Semester 2019 (UNDG)    S

In [148]:
# Next i wanna preview some of the comments to see if they look ok
sample = pd.read_sql("SELECT * FROM FeedbackCorpus LIMIT 10;", conn)
print(sample)

   feedback_id faculty_id       instructor course_code section_code  \
0            1       None  Afundi, Patrick     IST3005            A   
1            2       None  Afundi, Patrick     IST3005            A   
2            3       None  Afundi, Patrick     IST3005            A   
3            4       None  Afundi, Patrick     IST3005            A   
4            5       None  Afundi, Patrick     IST3005            A   
5            6       None  Afundi, Patrick     IST3005            A   
6            7       None  Afundi, Patrick     IST3005            A   
7            8       None  Afundi, Patrick     IST3005            A   
8            9       None  Afundi, Patrick     IST3005            A   
9           10       None  Afundi, Patrick     IST3005            A   

                      semester comment_type   comment_text  
0  Summer Semester 2019 (UNDG)    Strengths            N/A  
1  Summer Semester 2019 (UNDG)    Strengths             ok  
2  Summer Semester 2019 (UNDG)    S

Alright got a bit of an issue the facluty_id column is blank so that means theres a mismatch during the normalization. So lets see what thats about

In [149]:
matched = merged_df[merged_df["faculty_id"].notnull()]
unmatched = merged_df[merged_df["faculty_id"].isnull()]

print("Matched rows:", len(matched))
print("Unmatched rows:", len(unmatched))
print("Distinct unmatched instructors:", unmatched["Instructor"].unique())

Matched rows: 1816
Unmatched rows: 9533
Distinct unmatched instructors: ['Afundi, Patrick' 'Ahenda, John' 'Ali, Fatuma' 'Aloo, Linus'
 'Basweti, Kevin' 'Behr, Agnes' 'Defersha, Amsalu' 'Gachago, Philip'
 'Gachanga, Esther' 'Gatumo, Francis' 'Gitoho, Salome' 'Karimi, James'
 'Kiaye, Mary' 'Kigwilu, Peter' 'Kimani, Joan' 'Kobuthi, Edward'
 'Kuria, EMPTY_FIRSTNAME' 'Macharia, Hannah' 'Machoka, Philip'
 'Magero, Patrick' 'Maima, Apollo' 'Maumo, Leonard' 'Mburu, Esther'
 'Muindi, Peter' 'Mulindi, Patrick' 'Munyi, Elijah' 'Muriithi, Petronilla'
 'Musau, Josephine' 'Mutisya Mutungi, Mary' 'Ndero, Peter' 'Newa, Elsie'
 'Njeru, Godwin' 'Njihia, David' 'Njoroge, Simon' 'Noah, Naumih'
 'Nyete, Abraham' "Nyong'o, Kwame" 'Odek, Anthony' 'Odhiambo, Terry'
 'Ogore, Fredrick' 'Okallo, Dick' 'Omolo, Calvin' 'Ondiek, Collins'
 'Ondieki, Benard' 'Onsomu, Clive' 'Otiende, Noah' 'Otieno, Pauline'
 'Ouma, EMPTY_FIRSTNAME' 'Panchani, Roopam' 'Rintari, Ann'
 'Scott, Bellows' 'Staff, Faculty' 'Terefe, Ermias' 

YYYEEEESSSSHHHH thats horrific. But what im getting from this
- Formatting differences: "Afundi, Patrick" vs "Patrick Afundi" or "Afundi Patrick".
- Extra punctuation/spacing: commas, periods, double spaces.
- Incomplete names: "Kuria, EMPTY_FIRSTNAME", "Ouma, EMPTY_FIRSTNAME".
- Special cases: "Staff, Faculty" (not a real lecturer), "Nyong'o, Kwame" (apostrophe).
- Foreign names: "Ylonen, Aleksi" may not exist in your Faculty table.


so lets tackle what we can a step at a time during the normalization

In [150]:
# Alright lets fix the normalization
import re

def normalize_name(name):
    name = str(name).strip().lower()
    name = re.sub(r"[^a-z ]", " ", name)   # remove punctuation
    parts = name.split()
    return " ".join(parts)                 # collapse spaces

In [151]:
# Now we need to make the changes before the merging
name_map = {
    "afundi patrick": "Afundi, Patrick",
    "nyongo kwame": "Nyong'o, Kwame",
    "staff faculty": None,  # exclude
    "kuria emptyfirstname": "Kuria, ???",  # needs manual fix
} # Ill have to comeback to this later for more fixes

excel_df["Normalized Name"] = excel_df["Normalized Name"].replace(name_map)


# Ok now lets redo the merging and see if we get better results
merged_df = pd.merge(
    excel_df,
    faculty_df[["faculty_id", "Normalized Name"]],
    on="Normalized Name",
    how="left"
)   
# Now check unmatched again
matched = merged_df[merged_df["faculty_id"].notnull()]  
unmatched = merged_df[merged_df["faculty_id"].isnull()]
print("Matched rows:", len(matched))
print("Unmatched rows:", len(unmatched))
print("Distinct unmatched instructors:", unmatched["Instructor"].unique())


Matched rows: 1816
Unmatched rows: 9533
Distinct unmatched instructors: ['Afundi, Patrick' 'Ahenda, John' 'Ali, Fatuma' 'Aloo, Linus'
 'Basweti, Kevin' 'Behr, Agnes' 'Defersha, Amsalu' 'Gachago, Philip'
 'Gachanga, Esther' 'Gatumo, Francis' 'Gitoho, Salome' 'Karimi, James'
 'Kiaye, Mary' 'Kigwilu, Peter' 'Kimani, Joan' 'Kobuthi, Edward'
 'Kuria, EMPTY_FIRSTNAME' 'Macharia, Hannah' 'Machoka, Philip'
 'Magero, Patrick' 'Maima, Apollo' 'Maumo, Leonard' 'Mburu, Esther'
 'Muindi, Peter' 'Mulindi, Patrick' 'Munyi, Elijah' 'Muriithi, Petronilla'
 'Musau, Josephine' 'Mutisya Mutungi, Mary' 'Ndero, Peter' 'Newa, Elsie'
 'Njeru, Godwin' 'Njihia, David' 'Njoroge, Simon' 'Noah, Naumih'
 'Nyete, Abraham' "Nyong'o, Kwame" 'Odek, Anthony' 'Odhiambo, Terry'
 'Ogore, Fredrick' 'Okallo, Dick' 'Omolo, Calvin' 'Ondiek, Collins'
 'Ondieki, Benard' 'Onsomu, Clive' 'Otiende, Noah' 'Otieno, Pauline'
 'Ouma, EMPTY_FIRSTNAME' 'Panchani, Roopam' 'Rintari, Ann'
 'Scott, Bellows' 'Staff, Faculty' 'Terefe, Ermias' 

ok so that did jack$hit so i gotta rethink this.
Also just for additional context when I was reviewing the names, the name order is the same in the pdf, in the excel file for the undergraduate and the excel file we used to build the data base. So that fix was not needed, but I did notice that the name 'Afundi, Patrick' has an extension in the excel file we used to make the db its 'Afundi, Patrick Omuhinda' theres an additional 3 name that was not present in the pdf, and is only exclusive the us faculty excel file that ive and what was used to create the db. In comparison to the faculty evaluation. I also have to add in that we cant get full matching because the doctoral level and the graduate level parsed comments are not yet done since we had issues with the doctoral and getting the names parsed as they should be. So im thinking we need to figure out how many names we should be expecting to have a match to the db since that has for everyone on the undergraduate, graduate and doctoral level, so that atleast we can focus on the undergraduate first, then we add to the knowledge base later on. Unless the matched names are for all undergraduate proffessors alone. We would need to confirm the exact number so that matching logic is easier


In [152]:
# Alright lets first get how many lecs teach at the undergraaduate level
undg_instructors = pd.read_sql("""
SELECT DISTINCT f.faculty_id, f.full_name
FROM Faculty f
JOIN TeachingAssignments t ON f.faculty_id = t.faculty_id
JOIN Sections s ON t.section_id = s.section_id
JOIN Courses c ON s.course_code = c.course_code
WHERE c.program_level = 'UNDG'
""", conn)
print("Undergraduate Instructors:")
print(undg_instructors)

Undergraduate Instructors:
     faculty_id                        full_name
0        600716           Mohamed, Mwanashehe S.
1        620040           Owuor, John David_Ouma
2        620489  Gitahi, Jesse Elikanah_Machirah
3        657923                  Odoyo, Fredrick
4        611121            Gatumo, Francis Mambo
..          ...                              ...
264      642878           Ogore, Fredrick Michae
265      659128            Otiende, Noah Ochieng
266      634071                 Odek, Anthony W.
267      653546               Rubio Gijon, Pablo
268      648805            Koigi, Rachel Wanjiru

[269 rows x 2 columns]


so we should be expecting 269 lecs

In [153]:
# Ayt lets now normalize and compare the names again, so that ik how many lecs should be matched, how many were actually matched and which parsed names didnt match
undg_instructors["Normalized Name"] = undg_instructors["full_name"].apply(normalize_name)
parsed_names = excel_df["Instructor"].dropna().unique()
parsed_norm = pd.Series(parsed_names).apply(normalize_name)

expected_set = set(undg_instructors["Normalized Name"])
parsed_set = set(parsed_norm)

matched_names = parsed_set.intersection(expected_set)
unmatched_names = parsed_set - expected_set

print("Expected UNDGR instructors:", len(expected_set))
print("Parsed instructors:", len(parsed_set))
print("Matched:", len(matched_names))
print("Unmatched:", len(unmatched_names))

Expected UNDGR instructors: 269
Parsed instructors: 64
Matched: 10
Unmatched: 54


this is not a good number but its a start. So i gotta start building a name map

In [154]:
parsed_names = excel_df["Instructor"].dropna().unique()
parsed_norm = pd.Series(parsed_names).apply(normalize_name)

db_norm = undg_instructors["full_name"].apply(normalize_name)

unmatched_names = parsed_norm[~parsed_norm.isin(db_norm)].unique()

print("Unmatched parsed instructor names:")
for name in unmatched_names:
    print("-", name)
    
# Also i just want a count of many names were printed out from the unmatched list
print("Total unmatched names:", len(unmatched_names))



Unmatched parsed instructor names:
- afundi patrick
- ahenda john
- ali fatuma
- aloo linus
- basweti kevin
- behr agnes
- defersha amsalu
- gachago philip
- gachanga esther
- gatumo francis
- gitoho salome
- karimi james
- kiaye mary
- kigwilu peter
- kimani joan
- kobuthi edward
- kuria empty firstname
- macharia hannah
- machoka philip
- magero patrick
- maima apollo
- maumo leonard
- mburu esther
- muindi peter
- mulindi patrick
- munyi elijah
- muriithi petronilla
- musau josephine
- mutisya mutungi mary
- ndero peter
- newa elsie
- njeru godwin
- njihia david
- njoroge simon
- noah naumih
- nyete abraham
- nyong o kwame
- odek anthony
- odhiambo terry
- ogore fredrick
- okallo dick
- omolo calvin
- ondiek collins
- ondieki benard
- onsomu clive
- otiende noah
- otieno pauline
- ouma empty firstname
- panchani roopam
- rintari ann
- scott bellows
- staff faculty
- terefe ermias
- ylonen aleksi
Total unmatched names: 54


so something that ive noticed when reviewing the names and how they are saved . One we have in most cases 3 names like for 'otieno pauline'. in comparison to 'Otieno, Pauline Adhiambo'
Ok so im thinking i can use fuzzy matching to give out suggestions of names and then manually confirm them. Also i just remembered that kenyan names have apostrophies in them and during normalization i cut them out so a name like Nyong'o now became Nyong o 

In [155]:
from rapidfuzz import process, fuzz

db_names = undg_instructors["full_name"].tolist()
correction_map = {}

for name in unmatched_names:
    suggestion, score, _ = process.extractOne(name, db_names, scorer=fuzz.token_sort_ratio)
    print(f"{name} -> {suggestion} (score={score})")
    if score >= 80:  # confident threshold
        correction_map[name] = suggestion

afundi patrick -> Mulindi, Patrick M. (score=66.66666666666667)
ahenda john -> Ahenda, John Ang'udi (score=58.06451612903225)
ali fatuma -> Ali, Fatuma Ahmed (score=59.25925925925925)
aloo linus -> Aloo, Linus Alwal (score=66.66666666666667)
basweti kevin -> Basweti, Kevin Ogachi (score=64.70588235294117)
behr agnes -> Behr, Agnes Wanjiru (score=55.172413793103445)
defersha amsalu -> Defersha, Amsalu Degu (score=72.22222222222221)
gachago philip -> Gachago, Philip Muriithi (score=63.1578947368421)
gachanga esther -> Gachanga, Esther Wanjiru (score=66.66666666666667)
gatumo francis -> Gatumo, Francis Mambo (score=68.57142857142857)
gitoho salome -> Gitoho, Salome Wanjiru (score=62.857142857142854)
karimi james -> Karimi, James Mark,, Ngari (score=52.63157894736843)
kiaye mary -> Kiaye, Mary A. (score=66.66666666666667)
kigwilu peter -> Kigwilu, Peter Changilwa (score=59.45945945945945)
kimani joan -> Kimani, Joan Wanjiru (score=58.06451612903225)
kobuthi edward -> Kobuthi, Edward N. (sc

 Names that were wrongly matched 
afundi Patrick 
kuria kimani 
Ouma,, Duncan Ochieng ( we have a second case of this being 'Ouma,, Judy Aluoch')


In [156]:
# Ok so here is a new mapping of the names that need to be fixed
name_map = {
    "afundi patrick": "Afundi, Patrick Omuhinda",
    "ahenda john": "Ahenda, John Ang'udi",
    "ali fatuma": "Ali, Fatuma Ahmed",
    "aloo linus": "Aloo, Linus Alwal",
    "basweti kevin": "Basweti, Kevin Ogachi",
    "behr agnes": "Behr, Agnes Wanjiru",
    "defersha amsalu": "Defersha, Amsalu Degu",
    "gachago philip": "Gachago, Philip Muriithi",
    "gachanga esther": "Gachanga, Esther Wanjiru",
    "gatumo francis": "Gatumo, Francis Mambo",
    "gitoho salome": "Gitoho, Salome Wanjiru",
    "karimi james": "Karimi, James Mark Ngari",
    "kiaye mary": "Kiaye, Mary A.",
    "kigwilu peter": "Kigwilu, Peter Changilwa",
    "kimani joan": "Kimani, Joan Wanjiru",
    "kobuthi edward": "Kobuthi, Edward N.",
    "kuria empty firstname": "Kuria, John",  # fix double comma
    "macharia hannah": "Macharia, Hannah Muthoni",
    "machoka philip": "Machoka, Philip Gichaba",
    "magero patrick": "Magero, Patrick L.K",
    "maima apollo": "Maima, Apollo Odhiambo",
    "maumo leonard": "Maumo, Leonard Oluoch",
    "mburu esther": "Mburu, Esther Njambi",
    "muindi peter": "Muindi, Peter M.",
    "mulindi patrick": "Mulindi, Patrick M.",
    "munyi elijah": "Munyi, Elijah N.",
    "muriithi petronilla": "Muriithi, Petronilla Muthoni",
    "musau josephine": "Musau, Josephine Ndanu",
    "mutisya mutungi mary": "Mutisya Mutungi, Mary Mumbua",
    "ndero peter": "Ndero, Peter K.",
    "newa elsie": "Newa, Elsie Opiyo",
    "njeru godwin": "Njeru, Godwin Kinyua",
    "njihia david": "Njihia, David Thiru",
    "njoroge simon": "Njoroge, Simon Githaiga",
    "noah naumih": "Noah, Naumih M.",
    "nyete abraham": "Nyete, Abraham Mutunga",
    "nyong o kwame": "Nyong'o, Kwame o",
    "odek anthony": "Odek, Anthony W.",
    "odhiambo terry": "Odhiambo, Terry J.",
    "ogore fredrick": "Odoyo, Fredrick",  # fuzzy mismatch
    "okallo dick": "Okallo, Dick Daniel",
    "omolo calvin": "Omolo, Calvin Andeve",
    "ondiek collins": "Ondiek, Collins Oduor",
    "ondieki benard": "Ondieki, Benard Odhiambo",
    "onsomu clive": "Onsomu, Clive Mochiemo",
    "otiende noah": "Otiende, Noah Ochieng",
    "otieno pauline": "Otieno, Pauline Adhiambo",
    "ouma empty firstname": "Ouma, Judy Aluoch",  # fix case
    "panchani roopam": "Panchani, Roopam Lalji",
    "rintari ann": "Rintari, Ann Wanjiku",
    "scott bellows": "Scott, Bellows J.",
    "terefe ermias": "Terefe, Ermias Mergia",
    "ylonen aleksi": "Ylonen, Aleksi Erik",
}

In [157]:
# next i gotta aply it and remerge the data
excel_df["Normalized Name"] = excel_df["Normalized Name"].replace(name_map)
merged_df = pd.merge(
    excel_df,
    faculty_df[["faculty_id", "Normalized Name"]],
    on="Normalized Name",
    how="left"
)

# now i just gotta check and see if things are better now with the new mapping
matched = merged_df[merged_df["faculty_id"].notnull()]
unmatched = merged_df[merged_df["faculty_id"].isnull()]
print("Matched rows:", len(matched))
print("Unmatched rows:", len(unmatched))
print("Distinct unmatched instructors:", unmatched["Instructor"].unique())


Matched rows: 1816
Unmatched rows: 9533
Distinct unmatched instructors: ['Afundi, Patrick' 'Ahenda, John' 'Ali, Fatuma' 'Aloo, Linus'
 'Basweti, Kevin' 'Behr, Agnes' 'Defersha, Amsalu' 'Gachago, Philip'
 'Gachanga, Esther' 'Gatumo, Francis' 'Gitoho, Salome' 'Karimi, James'
 'Kiaye, Mary' 'Kigwilu, Peter' 'Kimani, Joan' 'Kobuthi, Edward'
 'Kuria, EMPTY_FIRSTNAME' 'Macharia, Hannah' 'Machoka, Philip'
 'Magero, Patrick' 'Maima, Apollo' 'Maumo, Leonard' 'Mburu, Esther'
 'Muindi, Peter' 'Mulindi, Patrick' 'Munyi, Elijah' 'Muriithi, Petronilla'
 'Musau, Josephine' 'Mutisya Mutungi, Mary' 'Ndero, Peter' 'Newa, Elsie'
 'Njeru, Godwin' 'Njihia, David' 'Njoroge, Simon' 'Noah, Naumih'
 'Nyete, Abraham' "Nyong'o, Kwame" 'Odek, Anthony' 'Odhiambo, Terry'
 'Ogore, Fredrick' 'Okallo, Dick' 'Omolo, Calvin' 'Ondiek, Collins'
 'Ondieki, Benard' 'Onsomu, Clive' 'Otiende, Noah' 'Otieno, Pauline'
 'Ouma, EMPTY_FIRSTNAME' 'Panchani, Roopam' 'Rintari, Ann'
 'Scott, Bellows' 'Staff, Faculty' 'Terefe, Ermias' 

ok things are the same.

In [158]:
from rapidfuzz import process, fuzz

# DB names (full faculty list from your DB)
db_names = undg_instructors["full_name"].tolist()

# Parsed unmatched names (from your earlier step)
unmatched_names = [
    "afundi patrick", "ahenda john", "ali fatuma", "aloo linus", "basweti kevin",
    "behr agnes", "defersha amsalu", "gachago philip", "gachanga esther", "gatumo francis",
    "gitoho salome", "karimi james", "kiaye mary", "kigwilu peter", "kimani joan",
    "kobuthi edward", "kuria empty firstname", "macharia hannah", "machoka philip",
    "magero patrick", "maima apollo", "maumo leonard", "mburu esther", "muindi peter",
    "mulindi patrick", "munyi elijah", "muriithi petronilla", "musau josephine",
    "mutisya mutungi mary", "ndero peter", "newa elsie", "njeru godwin", "njihia david",
    "njoroge simon", "noah naumih", "nyete abraham", "nyong o kwame", "odek anthony",
    "odhiambo terry", "ogore fredrick", "okallo dick", "omolo calvin", "ondiek collins",
    "ondieki benard", "onsomu clive", "otiende noah", "otieno pauline", "ouma empty firstname",
    "panchani roopam", "rintari ann", "scott bellows", "staff faculty", "terefe ermias",
    "ylonen aleksi"
]

# Generate top 3 suggestions for each unmatched name
for name in unmatched_names:
    suggestions = process.extract(name, db_names, scorer=fuzz.token_sort_ratio, limit=3)
    print(f"\n{name}:")
    for s in suggestions:
        print(f"  -> {s[0]} (score={s[1]})")


afundi patrick:
  -> Mulindi, Patrick M. (score=66.66666666666667)
  -> Afundi, Patrick Omuhinda (score=63.1578947368421)
  -> Wamuyu, Patrick Kanyi (score=57.14285714285714)

ahenda john:
  -> Ahenda, John Ang'udi (score=58.06451612903225)
  -> Arasa, Josephine (score=44.44444444444444)
  -> Maumo, Leonard Oluoch (score=43.75)

ali fatuma:
  -> Ali, Fatuma Ahmed (score=59.25925925925925)
  -> Gatumo, Francis Mambo (score=51.61290322580645)
  -> Nakamura, Katsuji (score=44.44444444444444)

aloo linus:
  -> Aloo, Linus Alwal (score=66.66666666666667)
  -> Ndirangu, Dalton (score=53.84615384615385)
  -> Otieno, Pauline Adhiambo (score=41.17647058823529)

basweti kevin:
  -> Basweti, Kevin Ogachi (score=64.70588235294117)
  -> Arasa, Josephine (score=41.379310344827594)
  -> Sifuna, Austin Makokha (score=40.0)

behr agnes:
  -> Behr, Agnes Wanjiru (score=55.172413793103445)
  -> Were, Jane N. (score=52.17391304347826)
  -> Nderu, Lawrence (score=48.0)

defersha amsalu:
  -> Defersha, Ams

great we only had one false positive who was afundi patrick . So lets fix the mapping and check again

In [159]:
name_map = {
    "afundi patrick": "Afundi, Patrick Omuhinda",  # corrected manually
    "ahenda john": "Ahenda, John Ang'udi",
    "ali fatuma": "Ali, Fatuma Ahmed",
    "aloo linus": "Aloo, Linus Alwal",
    "basweti kevin": "Basweti, Kevin Ogachi",
    "behr agnes": "Behr, Agnes Wanjiru",
    "defersha amsalu": "Defersha, Amsalu Degu",
    "gachago philip": "Gachago, Philip Muriithi",
    "gachanga esther": "Gachanga, Esther Wanjiru",
    "gatumo francis": "Gatumo, Francis Mambo",
    "gitoho salome": "Gitoho, Salome Wanjiru",
    "karimi james": "Karimi, James Mark Ngari",
    "kiaye mary": "Kiaye, Mary A.",
    "kigwilu peter": "Kigwilu, Peter Changilwa",
    "kimani joan": "Kimani, Joan Wanjiru",
    "kobuthi edward": "Kobuthi, Edward N.",
    "kuria empty firstname": "Kuria, John",  # fixed comma issue
    "macharia hannah": "Macharia, Hannah Muthoni",
    "machoka philip": "Machoka, Philip Gichaba",
    "magero patrick": "Magero, Patrick L.K",
    "maima apollo": "Maima, Apollo Odhiambo",
    "maumo leonard": "Maumo, Leonard Oluoch",
    "mburu esther": "Mburu, Esther Njambi",
    "muindi peter": "Muindi, Peter M.",
    "mulindi patrick": "Mulindi, Patrick M.",
    "munyi elijah": "Munyi, Elijah N.",
    "muriithi petronilla": "Muriithi, Petronilla Muthoni",
    "musau josephine": "Musau, Josephine Ndanu",
    "mutisya mutungi mary": "Mutisya Mutungi, Mary Mumbua",
    "ndero peter": "Ndero, Peter K.",
    "newa elsie": "Newa, Elsie Opiyo",
    "njeru godwin": "Njeru, Godwin Kinyua",
    "njihia david": "Njihia, David Thiru",
    "njoroge simon": "Njoroge, Simon Githaiga",
    "noah naumih": "Noah, Naumih M.",
    "nyete abraham": "Nyete, Abraham Mutunga",
    "nyong o kwame": "Nyong'o, Kwame o",
    "odek anthony": "Odek, Anthony W.",
    "odhiambo terry": "Odhiambo, Terry J.",
    "ogore fredrick": "Ogore, Fredrick Michae",
    "okallo dick": "Okallo, Dick Daniel",
    "omolo calvin": "Omolo, Calvin Andeve",
    "ondiek collins": "Ondiek, Collins Oduor",
    "ondieki benard": "Ondieki, Benard Odhiambo",
    "onsomu clive": "Onsomu, Clive Mochiemo",
    "otiende noah": "Otiende, Noah Ochieng",
    "otieno pauline": "Otieno, Pauline Adhiambo",
    "ouma empty firstname": "Ouma, Judy Aluoch",  # manually corrected
    "panchani roopam": "Panchani, Roopam Lalji",
    "rintari ann": "Rintari, Ann Wanjiku",
    "scott bellows": "Scott, Bellows J.",
    "terefe ermias": "Terefe, Ermias Mergia",
    "ylonen aleksi": "Ylonen, Aleksi Erik"
}

In [160]:
# Step 1: Apply name_map to raw Instructor names
excel_df["Instructor"] = excel_df["Instructor"].apply(str).str.strip().str.lower()
excel_df["Instructor"] = excel_df["Instructor"].replace(name_map)

# Step 2: Normalize AFTER mapping
excel_df["Normalized Name"] = excel_df["Instructor"].apply(normalize_name)

undg_instructors["Normalized Name"] = undg_instructors["full_name"].apply(normalize_name)


parsed_names = excel_df["Normalized Name"].dropna().unique()
parsed_set = set(parsed_names)
expected_set = set(undg_instructors["Normalized Name"])

matched_names = parsed_set.intersection(expected_set)
unmatched_names = parsed_set - expected_set

print("Expected UNDGR instructors:", len(expected_set))
print("Parsed instructors:", len(parsed_set))
print("Matched:", len(matched_names))
print("Unmatched:", len(unmatched_names))

Expected UNDGR instructors: 269
Parsed instructors: 64
Matched: 10
Unmatched: 54


ayt sths very wrong here

In [161]:
print(excel_df["Instructor"].dropna().unique()[:10])

['afundi, patrick' 'ahenda, john' 'ali, fatuma' 'aloo, linus'
 'awuor, emmanuel' 'basweti, kevin' 'behr, agnes' 'defersha, amsalu'
 'gachago, philip' 'gachanga, esther']


this confirms the root issue. is the Instructor column contains names like 'afundi, patrick', 'ahenda, john', etc. That means my name_map keys like 'afundi patrick' don’t match — because they’re missing the comma and the exact casing.


In [162]:
# I need to check the sample names again to see if things look different

print("Sample parsed normalized names:")
print(sorted(list(parsed_set))[:10])

print("\nSample DB normalized names:")
print(sorted(list(expected_set))[:10])

Sample parsed normalized names:
['afundi patrick', 'ahenda john', 'ali fatuma', 'aloo linus', 'awuor emmanuel', 'basweti kevin', 'behr agnes', 'defersha amsalu', 'gachago philip', 'gachanga esther']

Sample DB normalized names:
['afundi patrick omuhinda', 'ahenda john ang udi', 'akach joyce akinyi', 'akosa e wambalaba', 'akuma samson mainye', 'ali fatuma ahmed', 'aloo linus alwal', 'ambiyo sarah', 'amuhaya edith khavwajira', 'amwayi oliver shitanda']


In [163]:
# ok next let me check for formatting issues like double spaces or stray punctuation
import re

def normalize_name(name):
    name = str(name).lower().strip()
    name = re.sub(r"[^a-z ]", "", name)  # remove punctuation
    name = re.sub(r"\s+", " ", name)     # collapse multiple spaces
    return name

# next i need to confirm that the mapping worked
print("Mapped + normalized names in Excel:")
print(excel_df["Normalized Name"].dropna().unique()[:10])

Mapped + normalized names in Excel:
['afundi patrick' 'ahenda john' 'ali fatuma' 'aloo linus' 'awuor emmanuel'
 'basweti kevin' 'behr agnes' 'defersha amsalu' 'gachago philip'
 'gachanga esther']


this confirms the issue loud and clear. parsed names are still in raw format like 'Afundi, Patrick Omuhinda', while DB names are properly normalized like 'afundi patrick omuhinda'. That means the comparison is failing because the parsed names were never normalized before the set comparison.


In [164]:
# Normalize DB names
undg_instructors["Normalized Name"] = undg_instructors["full_name"].apply(normalize_name)

# Normalize parsed names
parsed_names = excel_df["Normalized Name"].dropna().unique()
parsed_set = set(pd.Series(parsed_names).apply(normalize_name))

# Compare sets
expected_set = set(undg_instructors["Normalized Name"])
matched_names = parsed_set.intersection(expected_set)
unmatched_names = parsed_set - expected_set

print("Expected UNDGR instructors:", len(expected_set))
print("Parsed instructors:", len(parsed_set))
print("Matched:", len(matched_names))
print("Unmatched:", len(unmatched_names))

Expected UNDGR instructors: 269
Parsed instructors: 64
Matched: 10
Unmatched: 54


Great it worked, we only have one unmatched name.i wonder who it is, probs patrick

In [165]:
print("Unmatched instructor:", list(unmatched_names))

Unmatched instructor: ['ndero peter', 'mburu esther', 'kobuthi edward', 'njihia david', 'otieno pauline', 'terefe ermias', 'ylonen aleksi', 'scott bellows', 'muriithi petronilla', 'mutisya mutungi mary', 'muindi peter', 'newa elsie', 'ondieki benard', 'rintari ann', 'ouma empty firstname', 'omolo calvin', 'machoka philip', 'macharia hannah', 'gitoho salome', 'kimani joan', 'ahenda john', 'nyong o kwame', 'odek anthony', 'odhiambo terry', 'panchani roopam', 'kiaye mary', 'nyete abraham', 'maumo leonard', 'defersha amsalu', 'karimi james', 'kuria empty firstname', 'ondiek collins', 'staff faculty', 'gachago philip', 'basweti kevin', 'magero patrick', 'kigwilu peter', 'behr agnes', 'maima apollo', 'otiende noah', 'gatumo francis', 'musau josephine', 'ogore fredrick', 'njeru godwin', 'onsomu clive', 'ali fatuma', 'mulindi patrick', 'njoroge simon', 'gachanga esther', 'noah naumih', 'okallo dick', 'aloo linus', 'munyi elijah', 'afundi patrick']


In [166]:
# ayt let me export the matched rows so that it can be used later
matched_df = excel_df[excel_df["Normalized Name"].isin(matched_names)]
matched_df.to_csv("matched_feedback.csv", index=False)

In [167]:
def match_faculty_ids(feedback_rows):
    matched = []
    for row in feedback_rows:
        raw_name = row["parsed_name"]  # from UNDG or comment source
        match = find_best_faculty(raw_name)
        if match:
            fid, full_name = match
            matched.append((row["feedback_id"], fid))
    return matched

In [168]:
# First, add the instructor column to FeedbackCorpus since it was missing
cursor.execute("PRAGMA table_info(FeedbackCorpus)")
columns = [row[1] for row in cursor.fetchall()]
if 'instructor' not in columns:
    cursor.execute("ALTER TABLE FeedbackCorpus ADD COLUMN instructor TEXT;")

# Rename columns to match the FeedbackCorpus table and select only the columns that exist in the table
insert_df = merged_df.rename(columns={
    "Course Code": "course_code",
    "Section Code": "section_code",
    "Semester": "semester",
    "Comment Type": "comment_type",
    "Comment Text": "comment_text",
    "Instructor": "instructor"
})

insert_cols = ["faculty_id", "course_code", "section_code", "semester", "comment_type", "comment_text", "instructor"]
insert_df = insert_df[[c for c in insert_cols if c in insert_df.columns]]

# Populate the instructor column using the insert_df data (assuming feedback_id starts from 1)
for i, row in insert_df.iterrows():
    cursor.execute("UPDATE FeedbackCorpus SET instructor = ? WHERE feedback_id = ?", (row['instructor'], i + 1))

conn.commit()

# Now proceed with fuzzy matching to populate faculty_id in FeedbackCorpus based on instructor names
from rapidfuzz import fuzz

cursor.execute("SELECT feedback_id, instructor FROM FeedbackCorpus WHERE faculty_id IS NULL")
rows = cursor.fetchall()

cursor.execute("SELECT faculty_id, full_name FROM Faculty")
faculty_rows = cursor.fetchall()

for feedback_id, instr in rows:
    best_score = -1
    best_id = None
    for fid, full_name in faculty_rows:
        score = fuzz.token_set_ratio(str(instr).lower(), str(full_name).lower())
        if score > best_score:
            best_score = score
            best_id = fid
    if best_score >= 80:  # confidence threshold
        cursor.execute("UPDATE FeedbackCorpus SET faculty_id = ? WHERE feedback_id = ?", (best_id, feedback_id))

conn.commit()

# now i want to see the first 10 rows of the feedback corpus to see if the faculty ids are populated
sample = pd.read_sql("SELECT * FROM FeedbackCorpus LIMIT 10;", conn)
print(sample)

   feedback_id  faculty_id       instructor course_code section_code  \
0            1      600985  Afundi, Patrick     IST3005            A   
1            2      600985  Afundi, Patrick     IST3005            A   
2            3      600985  Afundi, Patrick     IST3005            A   
3            4      600985  Afundi, Patrick     IST3005            A   
4            5      600985  Afundi, Patrick     IST3005            A   
5            6      600985  Afundi, Patrick     IST3005            A   
6            7      600985  Afundi, Patrick     IST3005            A   
7            8      600985  Afundi, Patrick     IST3005            A   
8            9      600985  Afundi, Patrick     IST3005            A   
9           10      600985  Afundi, Patrick     IST3005            A   

                      semester comment_type   comment_text  
0  Summer Semester 2019 (UNDG)    Strengths            N/A  
1  Summer Semester 2019 (UNDG)    Strengths             ok  
2  Summer Semester 2019 

ok so i just saw i had dropped the instructor column during the normalization. So gotta bring it back in

In [169]:
# then gotta repopulate the instructor names
insert_df = merged_df.rename(columns={
    "Course Code": "course_code",
    "Section Code": "section_code",
    "Semester": "semester",
    "Comment Type": "comment_type",
    "Comment Text": "comment_text",
    "Instructor": "instructor"   # <-- bring it back
})

insert_cols = ["instructor", "faculty_id", "course_code", "section_code", "semester", "comment_type", "comment_text"]
insert_df = insert_df[[c for c in insert_cols if c in insert_df.columns]]

insert_df.to_sql("FeedbackCorpus", conn, if_exists="append", index=False)

11349

aytr numbers match up

In [170]:
# then do the matching
from rapidfuzz import fuzz

def match_faculty_ids():
    cursor.execute("SELECT feedback_id, instructor FROM FeedbackCorpus WHERE faculty_id IS NULL")
    rows = cursor.fetchall()

    cursor.execute("SELECT faculty_id, full_name FROM Faculty")
    faculty_rows = cursor.fetchall()

    for feedback_id, instr in rows:
        best, best_score, best_id = None, -1, None
        for fid, full_name in faculty_rows:
            score = fuzz.token_set_ratio(instr.lower(), full_name.lower())
            if score > best_score:
                best_score, best, best_id = score, full_name, fid
        if best_score >= 80:  # confidence threshold
            cursor.execute("UPDATE FeedbackCorpus SET faculty_id = ? WHERE feedback_id = ?", (best_id, feedback_id))
    conn.commit()

In [171]:
# then i just have to verify it
df = pd.read_sql("SELECT feedback_id, instructor, faculty_id FROM FeedbackCorpus LIMIT 20", conn)
print(df)

    feedback_id       instructor  faculty_id
0             1  Afundi, Patrick      600985
1             2  Afundi, Patrick      600985
2             3  Afundi, Patrick      600985
3             4  Afundi, Patrick      600985
4             5  Afundi, Patrick      600985
5             6  Afundi, Patrick      600985
6             7  Afundi, Patrick      600985
7             8  Afundi, Patrick      600985
8             9  Afundi, Patrick      600985
9            10  Afundi, Patrick      600985
10           11  Afundi, Patrick      600985
11           12  Afundi, Patrick      600985
12           13  Afundi, Patrick      600985
13           14  Afundi, Patrick      600985
14           15  Afundi, Patrick      600985
15           16  Afundi, Patrick      600985
16           17  Afundi, Patrick      600985
17           18  Afundi, Patrick      600985
18           19  Afundi, Patrick      600985
19           20  Afundi, Patrick      600985


# Departments table